<a href="https://colab.research.google.com/github/SEOUL-ABSS/SHIPSHIP/blob/main/SONAR7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ShipsEar Ship-vs-Noise (1s) — V0a/V0b/V5~V8 (RAM-safe, short)
print("Setup...")
!pip -q install "tensorflow==2.19.0" tensorflow_hub==0.16.1 librosa==0.10.2.post1 soundfile==0.12.1 scikit-learn==1.5.2 psutil==5.9.8 seaborn==0.13.2 joblib==1.4.2
!apt -yq install fonts-nanum >/dev/null

import os, re, random, math, time, json, glob, shutil, warnings
from collections import Counter, defaultdict, OrderedDict
import numpy as np, pandas as pd, psutil, soundfile as sf
import tensorflow as tf, tensorflow_hub as hub, librosa
from tensorflow.keras import mixed_precision
import matplotlib.pyplot as plt, seaborn as sns, matplotlib.font_manager as fm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score, average_precision_score, balanced_accuracy_score, top_k_accuracy_score, accuracy_score
from sklearn.model_selection import GroupShuffleSplit

warnings.filterwarnings("ignore", category=UserWarning)
SEED=42; np.random.seed(SEED); random.seed(SEED); tf.random.set_seed(SEED)
try:
    for g in tf.config.experimental.list_physical_devices('GPU'):
        tf.config.experimental.set_memory_growth(g, True)
except: pass
mixed_precision.set_global_policy("mixed_float16")
if os.path.exists('/usr/share/fonts/truetype/nanum/NanumGothic.ttf'):
    fm.fontManager.addfont('/usr/share/fonts/truetype/nanum/NanumGothic.ttf')
    plt.rc('font', family='NanumGothic'); plt.rcParams['axes.unicode_minus'] = False

def mem(): return f"{psutil.Process().memory_info().rss/1024**3:.2f} GB"

# Paths
BASE="/content"; SHIPSEAR_DRIVE="/content/drive/MyDrive/ShipsEar"; SHIPSEAR=f"{BASE}/ShipsEar_colab"
os.makedirs("results", exist_ok=True); os.makedirs("cache", exist_ok=True); os.makedirs("artifacts", exist_ok=True)
try:
    from google.colab import drive; drive.mount('/content/drive', force_remount=False); print("Drive mounted.")
except Exception as e: print("Not Colab or Drive:", e)

# Config
YAM_SR=16000; BINARY_MODE=True; POS_LABEL="Ship"
CFG=dict(seg_dur=1.0, ship_overlap=0.2, noise_overlap=0.0,
         vad_frame_sec=0.5, vad_hop_sec=0.25, vad_top_db=25.0,
         test_size=0.2, epochs=40, batch=32, lr=5e-4,
         max_seg_per_group_per_class=500, noise_jitter_sec=0.5,
         topk=1, cache_emb=True)

# Speed toggles
MAKE_PLOTS=False
SAVE_AP=False

VERSIONS=[
    dict(name="v0a_yamnet_zeroshot", type="zero"),
    dict(name="v0b_emb_logreg_basic", type="emb", classifier="logreg", pooling="meanstd", aug=None),
    dict(name="v5_meanstd_mlp_aug",  type="emb", classifier="mlp", pooling="meanstd", aug="light"),
    dict(name="v6_ft_mean_headonly", type="ft",  pooling="mean",    aug="light"),
    dict(name="v7_ft_meanstd_headonly", type="ft", pooling="meanstd", aug="light"),
    dict(name="v8_ft_meanstd_headonly_tinyLR", type="ft", pooling="meanstd", aug="light"),
]

# --- Data copy ---
print("Data...")
if os.path.exists(SHIPSEAR_DRIVE):
    if not os.path.exists(SHIPSEAR) or not os.listdir(SHIPSEAR):
        shutil.copytree(SHIPSEAR_DRIVE, SHIPSEAR, dirs_exist_ok=True)
        print(" - Copied ShipsEar")
    else: print(" - ShipsEar exists")
else: raise FileNotFoundError(f"ShipsEar drive not found: {SHIPSEAR_DRIVE}")

# --- Labeling / groups ---
KW={"A":["fishing","trawler","trawl","mussel","tug","dredger","dredge"],
    "B":["motorboat","motor boat","pilot","sailboat","sailing"],
    "C":["ferry","passenger"],
    "D":["oceanliner","ocean liner","ro-ro","roro","ro_ro","cargo","containership","container","tanker","bulk","liner","oceangoing"],
    "E":["background","noise","ambient","no_ship","noship","silence"]}
def resolve_class(path):
    txt=(os.path.basename(os.path.dirname(path))+" "+os.path.basename(path)).lower()
    for c,kws in (("E",KW["E"]),("A",KW["A"]),("B",KW["B"]),("C",KW["C"]),("D",KW["D"])):
        if any(k in txt for k in kws): return c
    m=re.search(r'\bclass[_\s-]*([abcde])\b', txt); return m.group(1).upper() if m else None
def group_key(path):
    stem=os.path.splitext(os.path.basename(path))[0]
    m=re.search(r'(\d{8}[_-]?\d{4})', stem) or re.search(r'(\d{4}[-_]\d{2}[-_]\d{2}[_-]?\d{2}[-_]?\d{2})', stem)
    if m: return m.group(1)
    parent=os.path.basename(os.path.dirname(path)); toks=re.split(r'[_\-]+', stem); pref="_".join(toks[:3]) if len(toks)>=3 else stem
    return f"{parent}:{pref}"

# --- VAD / segments ---
EPS=1e-12
def get_activity(file_path, top_db=25.0, frame_sec=0.5, hop_sec=0.25):
    try:
        with sf.SoundFile(file_path) as f:
            sr=f.samplerate; n=len(f); F=max(1,int(frame_sec*sr)); H=max(1,int(hop_sec*sr))
            max_db=-np.inf; pos=0
            while pos+F<=n:
                f.seek(pos); y=f.read(frames=F, dtype='float32', always_2d=False); y=y.mean(axis=1) if y.ndim>1 else y
                rms=float(np.sqrt(np.mean(y**2))+EPS); max_db=max(max_db, 20*np.log10(rms+EPS)); pos+=H
            if not np.isfinite(max_db): return [], []
            th=max_db-top_db; active=[]; in_act=False; cur=0.0; pos=0
            while pos+F<=n:
                f.seek(pos); y=f.read(frames=F, dtype='float32', always_2d=False); y=y.mean(axis=1) if y.ndim>1 else y
                db=20*np.log10(float(np.sqrt(np.mean(y**2))+EPS))
                t0=pos/sr; t1=(pos+F)/sr
                if db>=th:
                    if not in_act: in_act=True; cur=t0
                else:
                    if in_act: in_act=False; active.append((cur,t1))
                pos+=H
            if in_act: active.append((cur,n/sr))
            inactive=[]; last=0.0; dur=n/sr
            for s,e in active:
                if s>last: inactive.append((last,s)); last=e
            if last<dur: inactive.append((last,dur))
            return active, inactive
    except: return [], []

def spans_to_segs(spans, seg_dur, hop):
    segs=[]
    for s,e in spans:
        if e-s < seg_dur: continue
        st=s
        while st <= e - seg_dur + 1e-9:
            segs.append((float(st),)); st += hop
    return segs

def build_segments(root, cfg):
    seg_dur=cfg["seg_dur"]; hop_ship=seg_dur*(1-cfg["ship_overlap"]); hop_noise=seg_dur*(1-cfg["noise_overlap"])
    noise_jitter=cfg["noise_jitter_sec"]; cap=cfg["max_seg_per_group_per_class"]
    infos=[]; labels=[]; groups=[]; missing=0; per_gc=defaultdict(int); summary=defaultdict(int)
    for fp in glob.glob(os.path.join(root, "**", "*.wav"), recursive=True):
        c=resolve_class(fp)
        if c is None: missing+=1; continue
        try: info=sf.info(fp)
        except: continue
        gk=group_key(fp)
        if c in "ABCD":
            act,_=get_activity(fp, cfg["vad_top_db"], cfg["vad_frame_sec"], cfg["vad_hop_sec"]); spans=act; hop=hop_ship
        else:
            dur=info.frames/info.samplerate; spans=[(0.0,dur)]; hop=hop_noise
        segs=spans_to_segs(spans, seg_dur, hop); random.shuffle(segs)
        for (st,) in segs:
            if c=="E" and noise_jitter>0:
                j=random.uniform(-noise_jitter, noise_jitter)
                st=max(0.0, min(st+j, (info.frames/info.samplerate) - seg_dur))
            key=(gk,c)
            if cap and per_gc[key]>=cap: continue
            infos.append((fp, float(st), info.samplerate)); labels.append(c); groups.append(gk)
            per_gc[key]+=1; summary[c]+=1
    return infos, labels, groups, summary, missing

# --- Audio IO / augment / resample ---
_WAVE_CACHE=OrderedDict()
_WAVE_CACHE_BYTES=0
_MAX_CACHE_BYTES=256*1024*1024

def _cache_get(fp):
    arr=_WAVE_CACHE.get(fp)
    if arr is not None:
        _WAVE_CACHE.move_to_end(fp)
    return arr

def _cache_put(fp, arr):
    global _WAVE_CACHE_BYTES
    size=getattr(arr, "nbytes", None)
    if size is None:
        try: size=arr.size*arr.itemsize
        except: size=0
    _WAVE_CACHE[fp]=arr
    _WAVE_CACHE.move_to_end(fp)
    _WAVE_CACHE_BYTES += size
    while _WAVE_CACHE_BYTES > _MAX_CACHE_BYTES and len(_WAVE_CACHE)>1:
        k,v=_WAVE_CACHE.popitem(last=False)
        try: _WAVE_CACHE_BYTES -= v.nbytes
        except: pass
def safe_resample(y, sr0, sr1):
    if sr0==sr1: return y.astype(np.float32)
    try:
        import scipy.signal as spsig
        g=math.gcd(int(sr0),int(sr1)); up=int(sr1)//g; down=int(sr0)//g
        return spsig.resample_poly(y, up, down).astype(np.float32)
    except Exception:
        try: return librosa.resample(y.astype(np.float32), orig_sr=sr0, target_sr=sr1, res_type="fft").astype(np.float32)
        except Exception:
            new_len=int(round(len(y)*float(sr1)/float(sr0)))
            xp=np.arange(len(y)); x_new=np.linspace(0,len(y),new_len,endpoint=False)
            return np.interp(x_new, xp, y).astype(np.float32)

def load_segment_cached(info, seg_dur, target_sr=YAM_SR, rms_norm=True):
    fp, st, sr0 = info
    try:
        y_full = _cache_get(fp)
        if y_full is None:
            y_full, sr_read = sf.read(fp, dtype='float32', always_2d=False)
            if y_full.ndim>1: y_full = y_full.mean(axis=1)
            if sr_read != target_sr: y_full = safe_resample(y_full, sr_read, target_sr)
            _cache_put(fp, y_full)
        L = int(seg_dur*target_sr); start = int(st*target_sr)
        if start >= len(y_full): return None
        y = y_full[start : min(start+L, len(y_full))]
        if len(y) < L: y = np.pad(y, (0, L-len(y)), mode='constant')
        if rms_norm:
            rms=float(np.sqrt(np.mean(y**2))+1e-12); y *= (10**(-20/20))/rms
        return y.astype(np.float32)
    except Exception as e:
        print("ERR load:", e); return None

def load_segment(info, seg_dur, target_sr=YAM_SR, rms_norm=True):
    fp, st, sr0 = info
    try:
        start=int(st*sr0); num=int(seg_dur*sr0)
        with sf.SoundFile(fp, 'r') as f:
            remain=f.frames-start
            if remain<=0: return None
            num=min(num, remain)
        y,_=sf.read(fp, start=start, stop=start+num, dtype='float32', always_2d=False)
        if y is None: return None
        if y.ndim>1: y=y.mean(axis=1)
        if sr0!=target_sr: y=safe_resample(y, sr0, target_sr)
        if rms_norm:
            rms=float(np.sqrt(np.mean(y**2))+1e-12); y *= (10**(-20/20))/rms
        return y.astype(np.float32)
    except Exception as e:
        print("ERR load:", e); return None

def augment(y, sr, kind="light"):
    if y is None or kind!="light": return y
    y = y * (10**(random.uniform(-3,3)/20))
    sh = random.randint(-int(0.25*sr), int(0.25*sr))
    if sh>0: y=np.concatenate([np.zeros(sh, dtype=y.dtype), y[:-sh]])
    elif sh<0: y=np.concatenate([y[-sh:], np.zeros(-sh, dtype=y.dtype)])
    return y

# --- YAMNet embed / zero-shot ---
YAM_URL="https://tfhub.dev/google/yamnet/1"
def make_yam_infer():
    ship_idx=[]
    try:
        module=hub.load(YAM_URL)
        def infer(y): return module(tf.convert_to_tensor(y, tf.float32))
        _=infer(np.zeros(16000, np.float32)); print("[YAMNet] hub.load")
        try:
            path=module.class_map_path().numpy().decode("utf-8")
            df=pd.read_csv(path); col='display_name' if 'display_name' in df.columns else df.columns[-1]
            names=df[col].astype(str).str.lower().tolist()
            subs=["boat","ship","sail","sailing","ferry","cargo","tanker","submarine","motorboat","watercraft","water vehicle","ocean liner","yacht","kayak","canoe","rowboat","row","fishing"]
            ship_idx=[i for i,n in enumerate(names) if any(s in n for s in subs)]
        except Exception:
            pass
        return infer, ship_idx
    except Exception:
        layer=hub.KerasLayer(YAM_URL, trainable=False)
        def infer(y):
            t=tf.convert_to_tensor(y, tf.float32)
            try: return layer(t)
            except: return layer(tf.expand_dims(t,0))
        _=infer(np.zeros(16000, np.float32)); print("[YAMNet] KerasLayer")
        return infer, ship_idx

def _emb_from_out(out):
    emb=None
    if isinstance(out,(list,tuple)) and len(out)>=2: emb=out[1]
    elif isinstance(out,dict):
        emb=out.get("embeddings") or out.get("embedding")
        if emb is None:
            for v in out.values():
                if isinstance(v,dict):
                    emb=v.get("embeddings") or v.get("embedding")
                    if emb is not None: break
    if emb is None: return None
    t=tf.convert_to_tensor(emb)
    if t.shape.rank==3 and t.shape[0]==1: t=tf.squeeze(t,0)
    if t.shape.rank==1: t=tf.expand_dims(t,0)
    return t

def embed_one(infer, y, pooling="meanstd"):
    if y is None: return None
    try:
        t=_emb_from_out(infer(y))
        if t is None or t.shape.rank!=2 or int(t.shape[0])==0: return None
        if pooling=="mean":
            feat=tf.reduce_mean(t,axis=0)
        else:
            m=tf.reduce_mean(t,axis=0); s=tf.math.reduce_std(t,axis=0); feat=tf.concat([m,s],axis=0)
        return feat.numpy().astype(np.float32)
    except Exception as e:
        print("ERR embed:", e); return None

def embed_many(infos, infer, cfg, pooling="meanstd", aug=None, cache_key=None, show_every=4000):
    cache=None
    if cfg["cache_emb"] and cache_key:
        cache=f"cache/emb_{cache_key}.npz"
        if os.path.exists(cache):
            z=np.load(cache, allow_pickle=True); print(f" - cache {cache} | X:{z['X'].shape} keep:{z['keep'].shape}"); return z["X"], z["keep"]
    X=[]; keep=[]
    for i,info in enumerate(infos,1):
        y=load_segment_cached(info, cfg["seg_dur"], YAM_SR, True)
        if aug: y=augment(y, YAM_SR, aug)
        e=embed_one(infer, y, pooling)
        if e is not None: X.append(e); keep.append(i-1)
        if i%show_every==0: print(f"  ... {i}/{len(infos)} (mem {mem()})")
    X=np.asarray(X,np.float32); keep=np.array(keep,np.int64)
    if cache and X.size>0: np.savez_compressed(cache, X=X, keep=keep)
    if X.size==0: print(f"ERR: no embeddings for {len(infos)} segs")
    return X, keep


def yam_scores(infer, y):
    out=infer(y); sc=None
    if isinstance(out,(list,tuple)) and len(out)>=1: sc=out[0]
    elif isinstance(out,dict): sc=out.get('scores') or out.get('predictions')
    if sc is None: return None
    t=tf.convert_to_tensor(sc)
    if t.shape.rank==3 and t.shape[0]==1: t=tf.squeeze(t,0)
    if t.shape.rank==1: return t.numpy().astype(np.float32)
    return tf.reduce_mean(t,axis=0).numpy().astype(np.float32)


# --- Models (emb/logreg/mlp) ---
def build_mlp(in_dim, n_cls, lr):
    reg=tf.keras.regularizers.l2(1e-4)
    x=tf.keras.Input(shape=(in_dim,)); h=tf.keras.layers.BatchNormalization()(x)
    h=tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=reg)(h); h=tf.keras.layers.Dropout(0.5)(h)
    h=tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=reg)(h); h=tf.keras.layers.Dropout(0.4)(h)
    y=tf.keras.layers.Dense(n_cls, activation='softmax')(h)
    m=tf.keras.Model(x,y); m.compile(optimizer=tf.keras.optimizers.Adam(lr), loss='categorical_crossentropy', metrics=['accuracy']); return m

def train_eval_emb(version, Xtr, ytr, Xte, yte, classes, cfg):
    res={}
    if Xtr.size==0 or Xte.size==0: raise RuntimeError("[emb] empty features")
    if version.get("classifier")=="mlp":
        clf=build_mlp(Xtr.shape[-1], len(classes), cfg["lr"])
        cb=[tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True, monitor='val_loss'),
            tf.keras.callbacks.ReduceLROnPlateau(patience=4, factor=0.5, min_lr=1e-6)]
        ytrc=tf.keras.utils.to_categorical(ytr, num_classes=len(classes)); ytec=tf.keras.utils.to_categorical(yte, num_classes=len(classes))
        cw={c:len(ytr)/ (len(np.unique(ytr))*cnt) for c,cnt in Counter(ytr).items()}
        t0=time.time(); clf.fit(Xtr, ytrc, validation_data=(Xte,ytec), epochs=cfg["epochs"], batch_size=cfg["batch"], verbose=0, class_weight=cw, callbacks=cb)
        probs=clf.predict(Xte, verbose=0).astype(np.float32); pred=probs.argmax(1); path=f"artifacts/{version['name']}_mlp.keras"; clf.save(path); res["artifact"]=path; res["time_sec"]=time.time()-t0
    else:
        from sklearn.linear_model import LogisticRegression; from sklearn.svm import SVC; import joblib
        sc=StandardScaler().fit(Xtr); Xtr_s=sc.transform(Xtr); Xte_s=sc.transform(Xte); t0=time.time()
        if version.get("classifier")=="logreg":
            clf=LogisticRegression(max_iter=2000, class_weight="balanced", n_jobs=-1); clf.fit(Xtr_s, ytr); probs=clf.predict_proba(Xte_s); pred=probs.argmax(1)
        else:
            clf=SVC(C=2.0, kernel='rbf', probability=True, class_weight='balanced'); clf.fit(Xtr_s, ytr); probs=clf.predict_proba(Xte_s); pred=probs.argmax(1)
        res["time_sec"]=time.time()-t0; joblib.dump(clf, f"artifacts/{version['name']}_{version['classifier']}.joblib"); joblib.dump(sc, f"artifacts/{version['name']}_scaler.joblib"); res["artifact"]="artifacts/*"
    true=yte; res["acc"]=accuracy_score(true,pred); res["bal_acc"]=balanced_accuracy_score(true,pred); res["macroF1"]=f1_score(true,pred,average='macro')
    try:
        res["macroROC"]=roc_auc_score(tf.keras.utils.to_categorical(true, len(classes)), probs, average='macro', multi_class='ovr')
    except: res["macroROC"]=np.nan
    try: res["topk"]=top_k_accuracy_score(true, probs, k=cfg['topk'], labels=range(len(classes)))
    except: res["topk"]=np.nan
    ap={};
    for i,lab in enumerate(classes):
        yb=(true==i).astype(int)
        ap[lab]=float(average_precision_score(yb, probs[:,i])) if 0<yb.sum()<len(yb) else float("nan")
    res["ap_per_class"]=ap; res["cm"]=confusion_matrix(true,pred)
    if len(classes)==2:
        try: pos_idx=classes.index(POS_LABEL) if POS_LABEL in classes else 1; res["macroROC"]=roc_auc_score(true, probs[:,pos_idx])
        except: res["macroROC"]=np.nan
        res["topk"]=np.nan
    return res

# --- Split ---
def strat_group_split(y, groups, test_size=0.2, seed=SEED):
    n=len(y)
    if n<2 or len(set(y))<2:
        raise RuntimeError("[데이터 부족] 세그먼트 수가 너무 적거나 클래스가 2종 미만입니다.\n- SHIPSEAR_DRIVE 경로와 폴더/파일명을 재확인하세요.\n- 라벨 매핑 규칙(resolve_class)과 실제 폴더명이 맞는지 점검하세요.")
    gss=GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=seed)
    tr,te=next(gss.split(np.arange(n), y, groups)); return tr,te,"GroupShuffleSplit"

# --- Pipeline ---
def run_all(cfg=CFG, versions=VERSIONS):
    print("Build segments...")
    infos,labels,groups,summary,missing=build_segments(SHIPSEAR,cfg)
    print(f" - per-class: {dict(summary)} | missing: {missing}")
    if BINARY_MODE: labels=["Ship" if l in "ABCD" else "Noise" for l in labels]
    le=LabelEncoder(); y=le.fit_transform(labels); classes=list(le.classes_); g=np.array(groups)
    tr,te,method=strat_group_split(y,g,cfg["test_size"]); print(f"[Split] {method} | train={len(tr)} test={len(te)} groups {len(set(g[tr]))}/{len(set(g[te]))}")
    Xtr_i=[infos[i] for i in tr]; ytr=y[tr]; Xte_i=[infos[i] for i in te]; yte=y[te]
    print("YAMNet infer...", end=""); infer, ship_idx = make_yam_infer(); print(f" OK (ship_idx={len(ship_idx)})")

    feature_bank = {}
    def get_feats(tag, infos, pooling, aug):
        key = (tag, pooling, aug or 'none', cfg['seg_dur'], len(infos))
        if key not in feature_bank:
            X, keep = embed_many(
                infos, infer, cfg, pooling, aug,
                cache_key=f"{tag}_pool={pooling}_aug={(aug or 'none')}_seg={cfg['seg_dur']}s"
            )
            feature_bank[key] = (X, keep)
        return feature_bank[key]

    all_rows = []
    for v in versions:
        print(f"\n==== {v['name']} ====")
        if v["type"] in ("emb","ft"):
            pooling = v.get("pooling","meanstd")
            aug = v.get("aug", None)
            print(" - embeds (train)...", end=""); Xtr, kt = get_feats("train", Xtr_i, pooling, aug); ytr_v = ytr[kt]; print(f" OK {Xtr.shape} (mem {mem()})")
            print(" - embeds (test)...",  end=""); Xte, ke = get_feats("test",  Xte_i, pooling, None); yte_v = yte[ke]; print(f" OK {Xte.shape} (mem {mem()})")
            if Xtr.size == 0 or Xte.size == 0: raise RuntimeError(f"[{v['name']}] 임베딩 실패")
            if v["type"] == "emb":
                res = train_eval_emb(v, Xtr, ytr_v, Xte, yte_v, classes, cfg)
            else:
                res = train_eval_emb(dict(v, classifier="mlp"), Xtr, ytr_v, Xte, yte_v, classes, cfg)

        elif v["type"] == "zero":
            if not ship_idx:
                print(" - zero-shot skipped (no ship idx)")
                continue
            print(" - zero-shot scoring...", end="")
            def score_list(infos):
                s = []
                for info in infos:
                    yseg = load_segment_cached(info, cfg["seg_dur"], YAM_SR, True)
                    if yseg is None: continue
                    sc = yam_scores(infer, yseg)
                    if sc is None: continue
                    s.append(float(1.0 - np.prod(1.0 - sc[ship_idx])))
                return np.array(s, np.float32)

            s_tr = score_list(Xtr_i); s_te = score_list(Xte_i)
            keep_tr = np.where(~np.isnan(s_tr))[0]; keep_te = np.where(~np.isnan(s_te))[0]
            s_tr = s_tr[keep_tr]; ytr_v = ytr[keep_tr]; s_te = s_te[keep_te]; yte_v = yte[keep_te]
            pos_idx = classes.index(POS_LABEL) if POS_LABEL in classes else 1
            ytr_bin = (ytr_v == pos_idx).astype(int); yte_bin = (yte_v == pos_idx).astype(int)
            t_best = 0.5; f_best = -1.0
            for t in np.linspace(0, 1, 21):
                f = f1_score(ytr_bin, (s_tr >= t).astype(int), average='binary', zero_division=0)
                if f > f_best: f_best = f; t_best = float(t)
            pred = (s_te >= t_best).astype(int)
            res = dict(
                artifact="",
                time_sec=0.0,
                acc=accuracy_score(yte_bin, pred),
                bal_acc=balanced_accuracy_score(yte_bin, pred),
                macroF1=f1_score(yte_bin, pred, average='macro'),
                macroROC=(roc_auc_score(yte_bin, s_te) if len(np.unique(yte_bin)) == 2 else np.nan),
                topk=np.nan,
                ap_per_class={POS_LABEL: float(average_precision_score(yte_bin, s_te))},
                cm=confusion_matrix(yte_bin, pred),
            )
            print(" OK")

        else:
            print(" - unknown type; skip")
            continue

        row = dict(
            version=v['name'], type=v['type'],
            pooling=v.get('pooling','-'),
            classifier=(v.get('classifier','-') if v['type']=='emb' else 'mlp'),
            aug=(v.get('aug') or 'none'),
            acc=res["acc"], bal_acc=res["bal_acc"], macroF1=res["macroF1"],
            macroROC=res["macroROC"], topk=res["topk"],
            time_sec=res["time_sec"], artifact=res.get("artifact","")
        )
        all_rows.append((row, res))

        if MAKE_PLOTS:
            cm = res["cm"]; plt.figure(figsize=(5.2, 4.5))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
            plt.xlabel("예측"); plt.ylabel("실제"); plt.title(f"CM — {v['name']}"); plt.tight_layout()
            plt.savefig(f"results/cm_{v['name']}.png", dpi=150); plt.close()

        if SAVE_AP:
            with open(f"results/ap_{v['name']}.json", "w") as f:
                json.dump(res["ap_per_class"], f, indent=2)

    if not all_rows:
        print("No results. Check data path."); return
    df=pd.DataFrame([r[0] for r in all_rows]).sort_values(["macroF1","bal_acc","acc"], ascending=False)
    df.to_csv("results/summary.csv", index=False)
    print("\n[SUMMARY]"); print(df.to_string(index=False))
    with open("results/report.md","w", encoding="utf-8") as f:
        f.write("# Ship vs Noise — V0a/V0b/V5~V8 비교 요약\n")
        f.write("|version|type|pooling|classifier|aug|acc|bal_acc|macroF1|macroROC|topk|time_sec|artifact|\n")
        f.write("|---|---|---|---|---|---:|---:|---:|---:|---:|---:|---|\n")
        for _,row in df.iterrows():
            macroROC = np.nan if pd.isna(row['macroROC']) else row['macroROC']
            topk = np.nan if pd.isna(row['topk']) else row['topk']
            f.write(f"|{row['version']}|{row['type']}|{row['pooling']}|{row['classifier']}|{row['aug']}|{row['acc']:.4f}|{row['bal_acc']:.4f}|{row['macroF1']:.4f}|{macroROC:.4f}|{topk:.4f}|{row['time_sec']:.1f}|{row['artifact']}|\n")
        f.write("- 혼동행렬: results/cm_*.png\n\n- AP per class: results/ap_*.json\n")
    print("\n결과 파일: results/summary.csv, results/report.md, results/cm_*.png, results/ap_*.json, artifacts/*")

# Run
run_all(CFG, VERSIONS)
print("\n🎉 완료")


Setup...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 25.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.5.2 which is incompatible.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive mounted.
Data...
 - Copied ShipsEar
Build segments...
 - per-class: {'C': 5085, 'B': 3134, 'E': 1140, 'A': 1855, 'D': 1513} | missing: 0
[Split] GroupShuffleSplit | train=102

In [ ]:
# ================================ OOD 평가 모듈 =================================
# 이 블록은 기존 파이프라인에서 학습이 끝난 후에 붙여 실행하세요.
# 필요 전역: YAMNET_SAMPLE_RATE, CONFIG, yamnet, clf(학습된 분류기), le,
#            Xtr/ytr, Xte/yte, Xtr_info/Xte_info (option), BASE 경로
# ==============================================================================

import os, subprocess, random, math, gc, glob, re
import numpy as np
import pandas as pd
import soundfile as sf
import librosa, librosa.display
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc, average_precision_score, precision_recall_curve

# ---------- 1) Git에서 OOD 샘플 오디오 가볍게 수집 ----------
OOD_ROOT = f"{BASE}/ood_audio_corpus"
os.makedirs(OOD_ROOT, exist_ok=True)

OOD_REPOS = [
    # 소형 예제/테스트 오디오가 비교적 들어있는 경우가 많음
    ("https://github.com/openai/whisper.git",          "whisper"),
    ("https://github.com/pytorch/audio.git",           "torchaudio"),
    ("https://github.com/iver56/audiomentations.git",  "audiomentations"),
    ("https://github.com/huggingface/transformers.git","transformers"),
]

def clone_if_needed(url, name):
    dst = os.path.join(OOD_ROOT, name)
    if not os.path.exists(dst):
        try:
            subprocess.run(["git","clone","--depth","1",url,dst], check=True, capture_output=True)
            print(f" - OK: {url}")
        except Exception as e:
            print(f" - FAIL: {url} ({e})")
    else:
        print(f" - already exists: {url}")
    return dst

print("\n[OOD] 리포지토리 수집 ...")
repo_dirs = [clone_if_needed(u,n) for (u,n) in OOD_REPOS]

# 오디오 확장자 패턴(넓게 잡되 개수 제한)
EXTS = (".wav",".flac",".ogg",".mp3",".m4a",".aac",".wma",".aiff",".aif",".aifc",".au",".mp2",".opus")
def find_audio_files(roots, max_total=200):
    all_files=[]
    for r in roots:
        for ext in EXTS:
            all_files += glob.glob(os.path.join(r, "**", f"*{ext}"), recursive=True)
    # 너무 많은 경우 샘플링
    if len(all_files) > max_total:
        random.shuffle(all_files)
        all_files = all_files[:max_total]
    return all_files

ood_files = find_audio_files(repo_dirs, max_total=250)
print(f" - 수집된 OOD 원본 파일: {len(ood_files)}")

# ---------- 2) OOD 세그먼트(5초) 스트리밍 생성 ----------
def stream_segments_for_ood(file_path, seg_dur=5.0, stride=5.0, cap_per_file=6):
    """librosa.load 없이 스트리밍으로 5초 구간을 균일 스트라이드로 최대 cap만 추출"""
    segs=[]
    try:
        info = sf.info(file_path)
        total = info.frames
        sr    = info.samplerate
        if info.duration < seg_dur: return segs

        # 균일 스트라이드로 시작점 후보 생성
        starts = np.arange(0, info.duration - seg_dur + 1e-9, stride)
        random.shuffle(starts)
        for st in starts[:cap_per_file]:
            segs.append((file_path, float(st), sr))
    except:
        pass
    return segs

# 너무 많이 뽑지 않도록 전체 cap (예: 800 세그먼트)
OOD_GLOBAL_CAP = 800
ood_segments=[]
for f in ood_files:
    segs = stream_segments_for_ood(f, seg_dur=CONFIG["segment_duration"], stride=CONFIG["segment_duration"], cap_per_file=6)
    ood_segments.extend(segs)
    if len(ood_segments) >= OOD_GLOBAL_CAP: break
print(f" - 생성된 OOD 세그먼트: {len(ood_segments)}")

# ---------- 3) OOD 임베딩 ----------
def load_and_process_segment(info, duration, target_sr, rms_norm=True):
    file_path, start_time, orig_sr = info
    try:
        start = int(start_time*orig_sr); num = int(duration*orig_sr)
        y, _ = sf.read(file_path, start=start, stop=start+num, dtype='float32', always_2d=False)
        if y.ndim>1: y = y.mean(axis=1)
        if orig_sr != target_sr:
            y = librosa.resample(y, orig_sr=orig_sr, target_sr=target_sr, res_type="kaiser_fast")
        if rms_norm:
            rms = np.sqrt(np.mean(y**2))+1e-12
            y = y * ((10**(-20/20))/rms)
        return y
    except:
        return None

def yamnet_embed_batch(infos, seg_dur=5.0, batch=128):
    X=[]; rms_list=[]; kept=[]
    for i,info in enumerate(infos):
        y = load_and_process_segment(info, seg_dur, YAMNET_SAMPLE_RATE, rms_norm=True)
        if y is None: continue
        # RMS(정규화 전에)도 저장해 에너지 편향 분석
        y_raw = load_and_process_segment(info, seg_dur, YAMNET_SAMPLE_RATE, rms_norm=False)
        rms_list.append(float(np.sqrt(np.mean(y_raw**2))+1e-12) if y_raw is not None else np.nan)
        try:
            _, emb, _ = yamnet(y)
            if emb.shape[0] == 0: continue
            X.append(tf.reduce_mean(emb, axis=0).numpy())
            kept.append(info)
        except:
            continue
        if (i+1)%500==0:
            print(f"  OOD 임베딩 {i+1}/{len(infos)}...")
    return np.asarray(X, dtype=np.float32), np.asarray(rms_list), kept

print("\n[OOD] 임베딩 추출 ...")
Xood, rms_ood, kept_ood = yamnet_embed_batch(ood_segments, seg_dur=CONFIG["segment_duration"])
print(f" - Xood:{Xood.shape}")

if Xood.shape[0] == 0:
    print("경고: OOD 임베딩이 비었습니다. 리포 소스나 max_total, cap을 조정해보세요.")

# ---------- 4) 임계값 선택(검증셋 TPR=95%) & ID/OOD FPR 비교 ----------
# 학습에 사용한 train에서 validation을 분리(간단히 10% hold-out)
def split_val_from_train(Xtr, ytr_onehot, val_ratio=0.1, seed=42):
    n = len(Xtr)
    idx = np.arange(n)
    rng = np.random.RandomState(seed)
    rng.shuffle(idx)
    k = max(1, int(round(n*val_ratio)))
    val_idx = idx[:k]; tr_idx = idx[k:]
    return Xtr[tr_idx], ytr_onehot[tr_idx], Xtr[val_idx], ytr_onehot[val_idx]

Xtr_fit, ytr_fit, Xval, yval = split_val_from_train(Xtr, ytr, val_ratio=0.1, seed=SEED)

# 재학습 없이 clf를 재사용하되, val 확률만 새로 추정
p_val = clf.predict(Xval, verbose=0)
p_te  = clf.predict(Xte,  verbose=0)

ship_idx = list(le.classes_).index('ship')
yval_bin = (yval.argmax(1)==ship_idx).astype(int)
yte_bin  = (yte.argmax(1)==ship_idx).astype(int)

def select_threshold_by_tpr(y_true_bin, y_score, target_tpr=0.95):
    fpr, tpr, thr = roc_curve(y_true_bin, y_score)
    # TPR이 target에 가장 근접한 점의 threshold
    j = np.argmin(np.abs(tpr - target_tpr))
    return float(thr[j]), float(tpr[j]), float(fpr[j])

tau, tpr_at_tau, fpr_at_tau = select_threshold_by_tpr(yval_bin, p_val[:,ship_idx], target_tpr=0.95)
print(f"\n[임계값] TPR@val≈95% → τ={tau:.4f} (val TPR={tpr_at_tau:.3f}, val FPR={fpr_at_tau:.3f})")

# ID-테스트 FPR / OOD FPR
fpr_id  = float(((p_te[:,ship_idx] >= tau) & (yte_bin==0)).mean()) if len(yte_bin)>0 else float('nan')

p_ood = clf.predict(Xood, verbose=0) if Xood.shape[0]>0 else np.zeros((0,len(le.classes_)),dtype=np.float32)
fpr_ood = float((p_ood[:,ship_idx] >= tau).mean()) if p_ood.shape[0]>0 else float('nan')

print(f"[FPR] ID(Test) FPR@τ={fpr_id:.4f} | OOD FPR@τ={fpr_ood:.4f}")

# ---------- 5) 시각화: 확률 분포 / ROC-PR / 에너지 편향 ----------
# (a) 확률 히스토그램
plt.figure(figsize=(7,5))
sns.kdeplot(p_te[yte_bin==1, ship_idx], label="ID: ship", fill=True, alpha=0.3)
sns.kdeplot(p_te[yte_bin==0, ship_idx], label="ID: noise", fill=True, alpha=0.3)
if p_ood.shape[0]>0:
    sns.kdeplot(p_ood[:, ship_idx], label="OOD (others)", fill=True, alpha=0.3)
plt.axvline(tau, color='k', ls='--', label=f"τ={tau:.2f}")
plt.title("Ship 확률 분포(ID vs OOD)"); plt.xlabel("P(ship)"); plt.legend(); plt.grid(True, alpha=0.3); plt.show()

# (b) ROC/PR (ID 기준)
fpr_id_curve, tpr_id_curve, _ = roc_curve(yte_bin, p_te[:,ship_idx])
roc_auc_id = auc(fpr_id_curve, tpr_id_curve)
prec, rec, _ = precision_recall_curve(yte_bin, p_te[:,ship_idx])
auprc = average_precision_score(yte_bin, p_te[:,ship_idx])

plt.figure(figsize=(11,4))
plt.subplot(1,2,1)
plt.plot(fpr_id_curve, tpr_id_curve, lw=2, label=f"AUC={roc_auc_id:.3f}")
plt.plot([0,1],[0,1],'--',alpha=0.4)
plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("ROC (ID Test)"); plt.legend(); plt.grid(True, alpha=0.3)

plt.subplot(1,2,2)
plt.plot(rec, prec, lw=2)
plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"PR (ID Test), AUPRC={auprc:.3f}")
plt.grid(True, alpha=0.3)
plt.show()

# (c) 에너지 decile 별 FPR (ID-Noise vs OOD)
def segment_rms(info, seg_dur=5.0):
    y = load_and_process_segment(info, seg_dur, YAMNET_SAMPLE_RATE, rms_norm=False)
    if y is None: return np.nan
    return float(np.sqrt(np.mean(y**2))+1e-12)

# ID-Noise RMS와 확률
id_noise_idx = np.where(yte_bin==0)[0]
rms_id_noise = np.array([segment_rms(Xte_info[i], CONFIG["segment_duration"]) if 'Xte_info' in globals() else np.nan
                         for i in id_noise_idx])
prob_id_noise = p_te[id_noise_idx, ship_idx]

def fpr_by_rms_decile(rms_arr, prob_arr, tau, n_bins=10):
    valid = np.isfinite(rms_arr)
    rms_arr, prob_arr = rms_arr[valid], prob_arr[valid]
    if len(rms_arr) < 10:
        return None
    qs = np.quantile(rms_arr, np.linspace(0,1,n_bins+1))
    bins = np.digitize(rms_arr, qs[1:-1], right=True)
    out=[]
    for b in range(n_bins):
        m = (bins==b)
        if m.sum()==0: out.append(np.nan)
        else: out.append(float((prob_arr[m] >= tau).mean()))
    return out, qs

ood_rms = np.zeros(0);
if len(kept_ood)>0:
    ood_rms = np.array([segment_rms(info, CONFIG["segment_duration"]) for info in kept_ood])

res_id = fpr_by_rms_decile(rms_id_noise, prob_id_noise, tau, n_bins=10)
res_ood = (None, None)
if len(ood_rms)>0:
    res_ood = fpr_by_rms_decile(ood_rms, p_ood[:,ship_idx], tau, n_bins=10)

if res_id is not None:
    fpr_bins_id, qs_id = res_id
    plt.figure(figsize=(7,4))
    plt.plot(range(1,11), fpr_bins_id, marker='o', label='ID-Noise')
    if isinstance(res_ood[0], list):
        plt.plot(range(1,11), res_ood[0], marker='o', label='OOD')
    plt.xticks(range(1,11)); plt.xlabel("RMS decile (낮음→높음)")
    plt.ylabel(f"FPR@τ"); plt.title("에너지 구간별 FPR (낮을수록 좋음)")
    plt.grid(True, alpha=0.3); plt.legend(); plt.show()
else:
    print("RMS decile 분석을 위한 유효 표본이 부족합니다.")

print("\n[요약]")
print(f" - 임계값 τ(Val TPR≈95%): {tau:.3f}")
print(f" - FPR(ID-noise)@τ: {fpr_id:.4f}")
print(f" - FPR(OOD)@τ: {fpr_ood:.4f} (낮을수록 좋음)")
print(f" - ROC-AUC(ID test): {roc_auc_id:.3f}, AUPRC(ID test): {auprc:.3f}")
print(" - 그래프: 확률분포/ROC/PR/에너지-디사일 FPR으로, 에너지-편향 여부를 함께 점검")
# ==============================================================================
